## Getting ready...

In [1]:
import tensorflow as tf
from tensorflow import keras

2022-01-20 14:08:59.845480: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


## How to do it

### Creating a custom layer

In [2]:
class MyCustomDense(tf.keras.layers.Layer):
    # Initialize this class with the number of units
    def __init__(self, units):
        super(MyCustomDense, self).__init__()
        self.units = units
        
    # Define the weights and the bias
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='random_normal',
                                 trainable=True)
        
    # Applying this layer transformation to the input tensor
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
    # Function to retrieve the configuration
    def get_config(self):
        return {'units': self.units}

In [3]:
# Create an input layer
inputs = keras.Input((12, 4))

# Add an instance of MyCustomDense Layer
outputs = MyCustomDense(2)(inputs)

# Create a model
model = keras.Model(inputs, outputs)

# Get the model config
config = model.get_config()

2022-01-20 14:17:52.167137: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-01-20 14:17:52.219431: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-20 14:17:52.219800: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.785GHz coreCount: 40 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-01-20 14:17:52.219814: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-01-20 14:17:52.221442: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-01-20 14:17:52.221467: I tensorflow/stream_executor/plat

In [4]:
# Reload the model from config
new_model = keras.Model.from_config(config,
                                   custom_objects={'MyCustomDense': MyCustomDense})

### Creating a custom model

In [5]:
mnist = tf.keras.datasets.mnist
(X_mnist_train, y_mnist_train), (X_mnist_test, y_mnist_test) = mnist.load_data()

train_mnist_features = X_mnist_train/255
test_mnist_features = X_mnist_test/255

In [6]:
class MyMNISTModel(tf.keras.Model):
    def __init__(self, num_classes):
        super(MyMNISTModel, self).__init__(name='my_mnist_model')
        self.num_classes = num_classes
        
        self.flatten_1 = tf.keras.layers.Flatten()
        self.dropout = tf.keras.layers.Dropout(0.1)
        self.dense_1 = tf.keras.layers.Dense(50, activation='relu')
        
        self.dense_2 = tf.keras.layers.Dense(10, activation='softmax')
        
    def call(self, inputs, training=False):
        
        x = self.flatten_1(inputs)
        
        # Apply dropout only during the training phase
        x = self.dense_1(x)
        if training:
            x = self.dropout(x, training=training)
        return self.dense_2(x)

In [9]:
my_mnist_model = MyMNISTModel(10)
# Compile
my_mnist_model.compile(optimizer='sgd',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

# Train
my_mnist_model.fit(train_mnist_features, y_mnist_train, validation_data=(test_mnist_features, y_mnist_test),
                  epochs=10)

2022-01-20 14:28:55.143004: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-01-20 14:28:55.159447: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 4200250000 Hz
2022-01-20 14:28:55.327129: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11


Epoch 1/10
 241/1875 [==>...........................] - ETA: 1s - loss: 1.6060 - accuracy: 0.5520

2022-01-20 14:28:55.597112: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


1875/1875 [==============================] - 2s 717us/step - loss: 0.7331 - accuracy: 0.7965 - val_loss: 0.3722 - val_accuracy: 0.8985
Epoch 2/10
1875/1875 [==============================] - 1s 660us/step - loss: 0.3954 - accuracy: 0.8852 - val_loss: 0.3027 - val_accuracy: 0.9158
Epoch 3/10
1875/1875 [==============================] - 1s 675us/step - loss: 0.3352 - accuracy: 0.9036 - val_loss: 0.2662 - val_accuracy: 0.9242
Epoch 4/10
1875/1875 [==============================] - 1s 679us/step - loss: 0.3013 - accuracy: 0.9128 - val_loss: 0.2424 - val_accuracy: 0.9298
Epoch 5/10
1875/1875 [==============================] - 1s 678us/step - loss: 0.2771 - accuracy: 0.9203 - val_loss: 0.2245 - val_accuracy: 0.9351
Epoch 6/10
1875/1875 [==============================] - 1s 676us/step - loss: 0.2561 - accuracy: 0.9266 - val_loss: 0.2107 - val_accuracy: 0.9400
Epoch 7/10
1875/1875 [==============================] - 1s 671us/step - loss: 0.2435 - accuracy: 0.9287 - val_loss: 0.1994 - val_accura